# Homework I: Search Algorithms

In [1]:
import time
# You may use time module to keep track of the time of your algortihm.

### Problem Formulation

#### State

The initial state of the problem will be represented by a list as follows. Note that ´1´s represent pegs, ´2´s represent empty holes on the board and ´0´s represent inaccessible locations (these are outside the Solo-Test board).

```
[0, 0, 1, 1, 1, 0, 0,
 0, 0, 1, 1, 1, 0, 0,
 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 2, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1,
 0, 0, 1, 1, 1, 0, 0,
 0, 0, 1, 1, 1, 0, 0]
```

Also note that this is a list so it is 1-dimensional, but our actions will consider as if it has 2-dimensions, for example we cannot move a peg in index ´21´ or ´22´ left because that is the left end of the board. So we will need to check valid moves depending on the accessible locations and the edges of the board.

#### Problem Class
Below we define the class of our problem:


In [2]:
up, right, down, left = 0, 1, 2, 3

class SoloTest():
    
    def __init__(self, goal=1):
        # Initial state
        self.initial = [0, 0, 1, 1, 1, 0, 0,
                        0, 0, 1, 1, 1, 0, 0,
                        1, 1, 1, 1, 1, 1, 1,
                        1, 1, 1, 2, 1, 1, 1,
                        1, 1, 1, 1, 1, 1, 1,
                        0, 0, 1, 1, 1, 0, 0,
                        0, 0, 1, 1, 1, 0, 0]
        self.goal = goal
     
    # Goal test - if number of remaining pegs is less than the goal (default goal is 1 peg)
    def goal_test(self,state):
        return state.count(1) <= self.goal

    
    # Check moves if a move is valid
    def checkMove(self, i, state, direction):
        x = i % 7
        y = int(i / 7)
        
        if direction == up:
            if y > 1 and state[i - 7] == 1 and state[i - 14] == 2:
                return True
        elif direction == left:
            if x > 1 and state[i - 1] == 1 and state[i - 2] == 2:
                return True
        elif direction == right:
            if x < 5 and state[i + 1] == 1 and state[i + 2] == 2:
                return True
        elif direction == down:
            if y < 5 and state[i + 7] == 1 and state[i + 14] == 2:
                return True
        return False

            
    # Returns available actions for a state
    def actions(self, state):
        moves = []
        for i in range(len(state)):
            if state[i] == 1:
                if(self.checkMove(i, state, up)):
                    moves.append((i,up))
                elif(self.checkMove(i, state, right)):
                    moves.append((i,right))
                elif(self.checkMove(i, state, down)):
                    moves.append((i,down))
                elif(self.checkMove(i, state, left)):
                    moves.append((i,left))
                else:
                    isolated = True
        return moves
    
    
    # Transition model - returns the resulting state for a state and action
    def result(self, state, action):
        s = list(state)
        location = action[0]
        direction = action[1] # 0 = up, 1 = right, 2 = down, 3 = left
        s[location] = 2
        if direction == up:
            s[location - 7] = 2
            s[location - 14] = 1
        elif direction == right:
            s[location + 1] = 2
            s[location + 2] = 1
        elif direction == down:
            s[location + 7] = 2
            s[location + 14] = 1
        elif direction == left:
            s[location - 1] = 2
            s[location - 2] = 1
        return tuple(s)
    
    # Action cost - 1 per movement. Action cost is irrelevant in this case because there is a fixed number of actions to reach to the goal state
    def action_cost(self, state1, action, state2):
        return 1
    


Below are some functions that you may be useful when implementing your search algorithms
- **Node**: The node data structure for the search tree.
- **expand()**: Function for expanding the nodes in the frontier.
- **boardsolo**: A function to format the state for printing (e.g. print(boardsolo(node.state))

In [3]:
class Node:
    def __init__(self,state,parent=None,action=None,path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        if parent:
            self.depth = parent.depth + 1
        else: 
            self.depth = 0
            
    def path(self):
        ## Path method returns a path from the initial state to the node's state
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

        
            
def expand(problem, node):
    s = node.state
    nodes = []
    for action in problem.actions(s):
        s1 = problem.result(s,action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)


def boardsolo(board, fmt=(7 * '{} {} {} {} {} {} {}\n')):
    return fmt.format(*board).replace('0', '_')



### Some reminder notes
• EMPTY?(queue) returns true only if there are no more elements in the queue.

• POP(queue) removes the first element of the queue and returns it.

• INSERT(element, queue) inserts an element and returns the resulting queue.

• The last-in, first-out or LIFO queue, which pops the newest element of the queue.

## A Brief Discussion
 For SoloTest problem, depth-first search and A* are seemed optimal algorithms to apply. Contrarily, since we are not looking for any exceptional results, and there is no way to achieve final solution with different path cost, depth-limited and iterative deepening are weaker search algorithms to implement regarding our problem with compared to DFS and A* .

 For SoloTest problem we have always exact number of steps to reach the goal, which is 31. There are finite branching factors, and number of steps to the solution is fixed. DFS is optimal and complete, its space complexity is optimal. Although, its time complexity is exponential, for SoloTest problem it is not an issue to be taken into account. DLS algorithm is suitable for solving infinite-path problems, but SoloTest has no chance to have an infinite nodes when reaching the goal. Thus, it would be non-optimal and incomplete if we set its predetermined limit lower than the number of steps of SoloTest’s solution. Besides, a good depth limit would not be determined if we have not solve the problem or if number of steps to the solution was not precise. 

 IDDFS is complete and optimal, its memory requirement is modest. It combines advantages of DFS and BFS algorithms. However, there is no difference between DFS and IDDFS, because our problem solution is fixed. Iterative deepening would be plausible if our search space was larger and depth of the solution was unknown. 

 By defining a good heuristic function when implementing our problem into A* search algorithm, we can expand fewer nodes, by doing that time and cost efficiency is increased and A* still guarantees to reach the solution for SoloTest. Even tough its time complexity is exponential, and it keeps every node in memory, A* with a good heuristic function would still be preferable.

## a) Depth-First Search

Implement the depth-first search algorithm for the problem. Run the algorithm and report **path_cost**, **path_to_goal**, **nodes_expanded**, **max_search_depth**, **max_frontier_nodes** and **running_time** for the solution (check homework for more information). You can implement an additional function to gather these parameters and print them in a readable format.


In [29]:
def depth_first(problem):
    
    # The first step is to test whether this is a goal state. Apparently it is not, nonetheless checking is good.
    if problem.goal_test(problem.initial):
        print("Initial state is the solution!")
        return True
    else:
        node = Node(problem.initial)
        frontier = list(expand(problem, node))
        reached = []  # when written as "{problem.initial: node}" python gives unhashable type "list" error.
                      # So an empty list is created to keep track of visited nodes.
                      # Because there is no infinite loop chance for this specific problem (SoloTest), 
                      # the code should run without an error.
                      # Creating a dictionary for EightPuzzle problem did not give unhashable type "list" error, 
                      # on the contrary, SoloTest does give this error, for reasons yet unknown!
        frontiers = [] # a list to keep track of max_frontier_nodes
        
        while len(frontier) > 0:
            frontiers.append(len(frontier)) # adds lenght of current number of frontier one by one into the list(frontiers)
            node = frontier.pop()
            if problem.goal_test(node.state):
                print("Solved!")
                path_cost = len(reached) # in order to get path_cost
                print(f"The number of moves taken to reach the goal is {path_cost}.")

                nodes_expanded = len(frontier) + path_cost # to get the total number of nodes that are expanded
                print(f"The number of nodes that have been expanded is {nodes_expanded}.")

                all_nodes = frontier + reached # to get the maximum depth of the search tree
                depth_list = []
                for node in all_nodes:
                    node_depth = node.depth
                    depth_list.append(node_depth)        
                max_depth = max(depth_list)      
                print(f"The maximum depth of the search tree during the execution of the algorithm is {max_depth}.")
                max_frontier_nodes = max(frontiers)
                print(f"The maximum number of nodes that were available in the frontier during the execution of the algorithm is {max_frontier_nodes}.")
                return node                
            else:
                for child in expand(problem, node):                  
                    if not child in reached:                    
                        frontier.append(child)
                reached.append(node)
        return reached


In [30]:
st_problem = SoloTest()

%time res = depth_first(st_problem)

Solved!
The number of moves taken to reach the goal is 271075.
The number of nodes that have been expanded is 271183.
The maximum depth of the search tree during the execution of the algorithm is 31.
The maximum number of nodes that were available in the frontier during the execution of the algorithm is 112.
CPU times: user 13min 31s, sys: 1.97 s, total: 13min 33s
Wall time: 13min 38s


In [25]:
def report_results(result):    
    print("Sequence of states from the initial state to the goal is as following: ")
    sequences = []
    node = result
    sequences.append(node.state)
    while(node.parent):
        node = node.parent
        sequences.append(node.state)
    sequences.reverse()
    num = 0
    for state in sequences:
        print("Sequence",num)
        print("----------------------------")
        print(boardsolo(state))
        print("----------------------------")
        num += 1
        
report_results(res)


Sequence of states from the initial state to the goal is as following: 
Sequence 0
----------------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 2 1 1 1
1 1 1 1 1 1 1
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _

----------------------------
Sequence 1
----------------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 2 1 1 1
_ _ 1 2 1 _ _
_ _ 1 1 1 _ _

----------------------------
Sequence 2
----------------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 1 2 2 1
_ _ 1 2 1 _ _
_ _ 1 1 1 _ _

----------------------------
Sequence 3
----------------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 1 1 2 1
_ _ 1 2 2 _ _
_ _ 1 1 2 _ _

----------------------------
Sequence 4
----------------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 1 1 2 1
_ _ 1 2 2 _ _
_ _ 2 2 1 _ _

----------------------------
Sequence 5
----------------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 

## b) Depth-Limited Search

Implement the depth-limited search algorithm for the problem. Run the algorithm and report **path_cost**, **path_to_goal**, **nodes_expanded**, **max_search_depth**, **max_frontier_nodes** and **running_time** for the solution (check homework for more information). You can implement an additional function to gather these parameters and print them in a readable format.


In [20]:
def depth_limited(problem, limit):

    if problem.goal_test(problem.initial):
        print("Initial state is the solution!")
        return True
    else:
        node = Node(problem.initial)
        frontier = list(expand(problem, node))
        reached = [] 
        frontiers = [] # a list to keep track of max_frontier_nodes
        nodes_beyond_limit = []
        
        while len(frontier) > 0:
                frontiers.append(len(frontier)) # adds lenght of current number of frontier one by one into the list(frontiers)
                node = frontier.pop()
                if problem.goal_test(node.state):
                    print("Solved!")
                    path_cost = len(reached) # in order to get path_cost
                    print(f"The number of moves taken to reach the goal is {path_cost}.")

                    nodes_expanded = len(frontier) + path_cost # to get the total number of nodes that are expanded
                    print(f"The number of nodes that have been expanded is {nodes_expanded}.")

                    all_nodes = frontier + reached
                    depth_list = []
                    for node in all_nodes:
                        node_depth = node.depth
                        depth_list.append(node_depth)        
                    max_depth = max(depth_list)      
                    print(f"The maximum depth of the search tree during the execution of the algorithm is {max_depth}.")
                    max_frontier_nodes = max(frontiers)
                    print(f"The maximum number of nodes that were available in the frontier during the execution of the algorithm is {max_frontier_nodes}.")
                    return node                
                else:
                    for child in expand(problem, node):   
                        node_depth = child.depth
                        if not child in reached and node_depth <= limit:                    
                            frontier.append(child)
                        if node_depth > limit:
                            nodes_beyond_limit.append(child)
                    reached.append(node)

        print("Could not be solved within given depth limit!")
        path_cost = len(reached) # in order to get path_cost
        print(f"The number of moves taken while trying to reach the goal is {path_cost}.")

        nodes_expanded = len(frontier) + path_cost + len(nodes_beyond_limit) # to get the total number of nodes that are expanded
        print(f"The number of nodes that have been expanded is {nodes_expanded}.")

        all_nodes = frontier + reached
        depth_list = []
        for node in all_nodes:
            node_depth = node.depth
            depth_list.append(node_depth)        
        max_depth = max(depth_list)      
        print(f"The maximum depth of the search tree during the execution of the algorithm is {max_depth}.")
        max_frontier_nodes = max(frontiers)
        print(f"The maximum number of nodes that were available in the frontier during the execution of the algorithm is {max_frontier_nodes}.")
        
        return node
    

In [14]:
st_problem = SoloTest() # The algorithm cannot solve the problem within predetermined limit. 

%time res = depth_limited(st_problem, 6)

Could not be solved within given depth limit!
The number of moves taken while trying to reach the goal is 24824.
The number of nodes that have been expanded is 204685.
The maximum depth of the search tree during the execution of the algorithm is 6.
The maximum number of nodes that were available in the frontier during the execution of the algorithm is 33.
CPU times: user 56 s, sys: 169 ms, total: 56.2 s
Wall time: 56.4 s


In [15]:
def report_results(result):    
    print("Sequence of states from the initial state to the goal is as following: ")
    sequences = []
    node = result
    sequences.append(node.state)
    while(node.parent):
        node = node.parent
        sequences.append(node.state)
    sequences.reverse()
    num = 0
    for state in sequences:
        print("Sequence",num)
        print("----------------------------")
        print(boardsolo(state))
        print("----------------------------")
        num += 1
        
report_results(res)


Sequence of states from the initial state to the goal is as following: 
Sequence 0
----------------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 2 1 1 1
1 1 1 1 1 1 1
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _

----------------------------
Sequence 1
----------------------------
_ _ 1 1 1 _ _
_ _ 1 2 1 _ _
1 1 1 2 1 1 1
1 1 1 1 1 1 1
1 1 1 1 1 1 1
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _

----------------------------
Sequence 2
----------------------------
_ _ 1 1 1 _ _
_ _ 1 2 1 _ _
1 2 2 1 1 1 1
1 1 1 1 1 1 1
1 1 1 1 1 1 1
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _

----------------------------
Sequence 3
----------------------------
_ _ 2 1 1 _ _
_ _ 2 2 1 _ _
1 2 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 1 1 1 1
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _

----------------------------
Sequence 4
----------------------------
_ _ 1 2 2 _ _
_ _ 2 2 1 _ _
1 2 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 1 1 1 1
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _

----------------------------
Sequence 5
----------------------------
_ _ 1 2 2 _ _
_ _ 2 2 1 _ _
1 1 2 2 1 1 1
1 1 1 

In [50]:
st_problem = SoloTest() # DLS did exact same search with DFS, if we set the limit 31 for this specific problem,
                        # which is the number of steps to reach the goal.
                        # Its path to goal(step by step) in graphical format would be identical with DFS. 
%time res = depth_limited(st_problem, 31)

Solved!
The number of moves taken to reach the goal is 271075.
The number of nodes that have been expanded is 271183.
The maximum depth of the search tree during the execution of the algorithm is 31.
The maximum number of nodes that were available in the frontier during the execution of the algorithm is 112.
CPU times: user 13min 19s, sys: 2.15 s, total: 13min 21s
Wall time: 13min 31s


In [22]:
def report_results(result):    
    print("Sequence of states from the initial state to the goal is as following: ")
    sequences = []
    node = result
    sequences.append(node.state)
    while(node.parent):
        node = node.parent
        sequences.append(node.state)
    sequences.reverse()
    num = 0
    for state in sequences:
        print("Sequence",num)
        print("----------------------------")
        print(boardsolo(state))
        print("----------------------------")
        num += 1
        
report_results(res)


Sequence of states from the initial state to the goal is as following: 
Sequence 0
----------------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 2 1 1 1
1 1 1 1 1 1 1
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _

----------------------------
Sequence 1
----------------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 2 1 1 1
_ _ 1 2 1 _ _
_ _ 1 1 1 _ _

----------------------------
Sequence 2
----------------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 1 2 2 1
_ _ 1 2 1 _ _
_ _ 1 1 1 _ _

----------------------------
Sequence 3
----------------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 1 1 2 1
_ _ 1 2 2 _ _
_ _ 1 1 2 _ _

----------------------------
Sequence 4
----------------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 1 1 2 1
_ _ 1 2 2 _ _
_ _ 2 2 1 _ _

----------------------------
Sequence 5
----------------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 

## c) Iterative Deepening Depth-Limited Search

Implement the iterative deepening depth-limited search algorithm for the problem. Run the algorithm and report **path_cost**, **path_to_goal**, **nodes_expanded**, **max_search_depth**, **max_frontier_nodes** and **running_time** for the solution (check homework for more information). You can implement an additional function to gather these parameters and print them in a readable format.


In [17]:
def iter_deep_df(problem, limit):
        
        if problem.goal_test(problem.initial):
            print("Initial state is the solution!")
            return True
        else:
            node = Node(problem.initial)
            frontier = list(expand(problem, node))
            reached = [] 
            frontiers = [] # a list to keep track of max_frontier_nodes
            nodes_beyond_limit = []
      
            while len(frontier) > 0:
                frontiers.append(len(frontier)) # adds lenght of current number of frontier one by one into the list(frontiers)
                node = frontier.pop()
                if problem.goal_test(node.state):
                    print("Solution found!")
                    path_cost = len(reached) # in order to get path_cost
                    print(f"The number of moves taken to reach the goal is {path_cost}.")
 
                    nodes_expanded = len(frontier) + path_cost # to get the total number of nodes that are expanded
                    print(f"The number of nodes that have been expanded is {nodes_expanded}.")

                    all_nodes = frontier + reached
                    depth_list = []
                    for node in all_nodes:
                        node_depth = node.depth
                        depth_list.append(node_depth)        
                    max_depth = max(depth_list)      
                    print(f"The maximum depth of the search tree during the execution of the algorithm is {max_depth}.")
                    max_frontier_nodes = max(frontiers)
                    print(f"The maximum number of nodes that were available in the frontier during the execution of the algorithm is {max_frontier_nodes}.")
                    return node                
                else:
                    for child in expand(problem, node):   
                        node_depth = child.depth
                        if not child in reached and node_depth <= limit:                    
                            frontier.append(child)
                        if node_depth > limit:
                            nodes_beyond_limit.append(child)
                    reached.append(node)

            print("Solution not found!")
            path_cost = len(reached) # in order to get path_cost
            print(f"The number of moves taken while trying to reach the goal is {path_cost}.")

            nodes_expanded = len(frontier) + path_cost + len(nodes_beyond_limit) # to get the total number of nodes that are expanded
            print(f"The number of nodes that have been expanded is {nodes_expanded}.")

            all_nodes = frontier + reached
            depth_list = []
            for node in all_nodes:
                node_depth = node.depth
                depth_list.append(node_depth)        
            max_depth = max(depth_list)      
            print(f"The maximum depth of the search tree during the execution of the algorithm is {max_depth}.")
            max_frontier_nodes = max(frontiers)
            print(f"The maximum number of nodes that were available in the frontier during the execution of the algorithm is {max_frontier_nodes}.")
            print("------------------------------")
            print(f"Could not be solved within given depth limit which was {limit}")
            multiplier = limit + 1
            limit = limit * multiplier # Multiplier method is used for this problem,
                                       # because if the limit was to be increased by one each time, 
                                       # search time would be too long! It will eventually reach the goal 
                                       # when limit is 42 (1*2, 2*3, 6*7) knowing that the goal is reached at 31th
                                       # step, this formula is created specifically for SoloTest(). Limitation 
                                       # formula is implemented in order to deal with around 15th-16th step's 
                                       # number of frontier and countless possible actions etc.,
                                       # and it is used for lowering the algorithm's time complexity.
            print(f"Expanding the limit, multiplying it by {multiplier} for further search...")
            print("------------------------------")
            
            return iter_deep_df(problem, limit)
            return reached

             

In [61]:
st_problem = SoloTest()

%time res = iter_deep_df(st_problem, 1)

Solution not found!
The number of moves taken while trying to reach the goal is 4.
The number of nodes that have been expanded is 16.
The maximum depth of the search tree during the execution of the algorithm is 1.
The maximum number of nodes that were available in the frontier during the execution of the algorithm is 4.
------------------------------
Could not be solved within given depth limit which was 1
Expanding the limit, multiplying it by 2 for further search...
------------------------------
Solution not found!
The number of moves taken while trying to reach the goal is 16.
The number of nodes that have been expanded is 76.
The maximum depth of the search tree during the execution of the algorithm is 2.
The maximum number of nodes that were available in the frontier during the execution of the algorithm is 6.
------------------------------
Could not be solved within given depth limit which was 2
Expanding the limit, multiplying it by 3 for further search...
--------------------

In [65]:
def report_results(result):    
    print("Sequence of states from the initial state to the goal is as following: ")
    sequences = []
    node = result
    sequences.append(node.state)
    while(node.parent):
        node = node.parent
        sequences.append(node.state)
    sequences.reverse()
    num = 0
    for state in sequences:
        print("Sequence",num)
        print("----------------------------")
        print(boardsolo(state))
        print("----------------------------")
        num += 1
            
report_results(res)


Sequence of states from the initial state to the goal is as following: 
Sequence 0
----------------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 2 1 1 1
1 1 1 1 1 1 1
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _

----------------------------
Sequence 1
----------------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 2 1 1 1
_ _ 1 2 1 _ _
_ _ 1 1 1 _ _

----------------------------
Sequence 2
----------------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 1 2 2 1
_ _ 1 2 1 _ _
_ _ 1 1 1 _ _

----------------------------
Sequence 3
----------------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 1 1 2 1
_ _ 1 2 2 _ _
_ _ 1 1 2 _ _

----------------------------
Sequence 4
----------------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 1 1 2 1
_ _ 1 2 2 _ _
_ _ 2 2 1 _ _

----------------------------
Sequence 5
----------------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 

## d) A*

Implement the A* algorithm for the problem (with a heuristic). Run the algorithm and report **path_cost**, **path_to_goal**, **nodes_expanded**, **max_search_depth**, **max_frontier_nodes** and **running_time** for the solution (check homework for more information). You can implement an additional function to gather these parameters and print them in a readable format.

In [7]:
def h(problem,node):
    candidates = list(expand(problem, node))    
    if len(candidates) < 3:
        if node.depth < 28:
            return False
        else:
            return True
    else:
        return True   

In [8]:
def a_star(problem):
    
    if problem.goal_test(problem.initial):
        print("Initial state is the solution!")
        return True
    else:
        node = Node(problem.initial)
        frontier = list(expand(problem, node))
        reached = []                          
        frontiers = []
        bad_branch = [] # to keep track of ignored nodes
        
        while len(frontier) > 0:
            frontiers.append(len(frontier))
            node = frontier.pop()
            
            if problem.goal_test(node.state):
                print("Solved!")
                path_cost = len(reached) # in order to get path_cost
                print(f"The number of moves taken to reach the goal is {path_cost}.")

                nodes_expanded = len(frontier) + path_cost # to get the total number of nodes that are expanded
                print(f"The number of nodes that have been expanded is {nodes_expanded}.")

                all_nodes = frontier + reached # to get the maximum depth of the search tree
                depth_list = []
                for node in all_nodes:
                    node_depth = node.depth
                    depth_list.append(node_depth)        
                max_depth = max(depth_list)      
                print(f"The maximum depth of the search tree during the execution of the algorithm is {max_depth}.")
                max_frontier_nodes = max(frontiers)
                print(f"The maximum number of nodes that were available in the frontier during the execution of the algorithm is {max_frontier_nodes}.")
                ignored = len(bad_branch)
                print(f"Ignored node count: {ignored}.") # Number of ignored nodes which were discarded before
                                                         # expanding. Assumption is: there must be at least 3
                                                         # possible actions before depth 28. There might be other 
                                                         # solutions where possible action count is lower than 3
                                                         # before reaching depth = 28. Yet, opting to ignore such
                                                         # occasions in order to achieve a better performance(search speed)
                                                         # seemed reasonably feasible.
                return node
            else:
                for child in expand(problem, node):    
                    node_depth = child.depth
                    if not child in reached:
                        if h(problem, child):                            
                            frontier.append(child)
                        else:
                            bad_branch.append(child)
                reached.append(node)
        return reached         
    

In [9]:
st_problem = SoloTest()
%time res = a_star(st_problem)

Solved!
The number of moves taken to reach the goal is 46635.
The number of nodes that have been expanded is 46739.
The maximum depth of the search tree during the execution of the algorithm is 31.
The maximum number of nodes that were available in the frontier during the execution of the algorithm is 110.
Ignored node count: 42450.
CPU times: user 58.9 s, sys: 240 ms, total: 59.1 s
Wall time: 59.7 s


In [16]:
def report_results(result):    
    print("Sequence of states from the initial state to the goal is as following: ")
    sequences = []
    node = result
    sequences.append(node.state)
    while(node.parent):
        node = node.parent
        sequences.append(node.state)
    sequences.reverse()
    num = 0
    for state in sequences:
        print("Sequence",num)
        print("----------------------------")
        print(boardsolo(state))
        print("----------------------------")
        num += 1
            
report_results(res)


Sequence of states from the initial state to the goal is as following: 
Sequence 0
----------------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 2 1 1 1
1 1 1 1 1 1 1
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _

----------------------------
Sequence 1
----------------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 2 1 1 1
_ _ 1 2 1 _ _
_ _ 1 1 1 _ _

----------------------------
Sequence 2
----------------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 1 2 2 1
_ _ 1 2 1 _ _
_ _ 1 1 1 _ _

----------------------------
Sequence 3
----------------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 1 1 2 1
_ _ 1 2 2 _ _
_ _ 1 1 2 _ _

----------------------------
Sequence 4
----------------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 1 1 2 1
_ _ 1 2 2 _ _
_ _ 2 2 1 _ _

----------------------------
Sequence 5
----------------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 